In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.CDSSMPreprocessor(fixed_length_left=10, fixed_length_right=40)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 6532.57it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:05<00:00, 3734.87it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3414564.50it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 9072.20it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4324.55it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 74492.74it/s]
Processing text_left with chain_transform of NgramLetterUnit => WordHashingUnit: 100

In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 3838.95it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 3971.28it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 73029.28it/s]
Processing text_left with chain_transform of NgramLetterUnit => WordHashingUnit: 100%|██████████| 122/122 [00:00<00:00, 6352.17it/s]
Processing text_right with chain_transform of NgramLetterUnit => WordHashingUnit: 100%|██████████| 1115/1115 [00:00<00:00, 2257.86it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9320.59it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2300/

In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.CDSSMModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['filters'] = 50
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 'same'
model.params['conv_activation_func'] = 'relu'
model.params['w_initializer'] = 'glorot_normal'
model.params['b_initializer'] = 'zeros'
model.params['mlp_num_layers'] = 0
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 10
model.params['mlp_activation_func'] = 'linear'
model.params['dropout_rate'] = 0.9
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
#model.backend.summary()

Parameter "name" set to CDSSMModel.


In [8]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=5, num_neg=1, batch_size=20)

In [9]:
history = model.fit_generator(train_generator, epochs=10, callbacks=[evaluate], workers=1, use_multiprocessing=False)

Epoch 1/10
255/255 [==============================] - 59s 230ms/step - loss: 0.7897
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.430714 - normalized_discounted_cumulative_gain@5(0):0.523023 - mean_average_precision(0):0.470724
Epoch 2/10
255/255 [==============================] - 51s 201ms/step - loss: 0.5914
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.493252 - normalized_discounted_cumulative_gain@5(0):0.575607 - mean_average_precision(0):0.522119
Epoch 3/10
255/255 [==============================] - 55s 216ms/step - loss: 0.4750
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.490502 - normalized_discounted_cumulative_gain@5(0):0.561662 - mean_average_precision(0):0.515332
Epoch 4/10
255/255 [==============================] - 53s 209ms/step - loss: 0.3961
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.497243 - normalized_discounted_cumulative_gain@5(0):0.559326 - mean_average_precision(0):0.513827
